# 고객요청의도 분석 후 활용하기 (인증서, 공인인증서)

# LangChain 라이브러리 설치

In [ ]:
!pip install langchain openai chromadb tiktoken pypdf unstructured sentence-transformers langchain-community

In [1]:
import pandas as pd

# 데이터 읽어오기

In [ ]:
json_file_path = '민원(콜센터) 질의응답_금융보험_잔고 및 거래내역_Training.json'
df = pd.read_json(json_file_path)
df

In [3]:
len(df)

74358

In [4]:
df.columns

Index(['도메인', '카테고리', '대화셋일련번호', '화자', '문장번호', '고객의도', '상담사의도', 'QA',
       '고객질문(요청)', '상담사질문(요청)', '고객답변', '상담사답변', '개체명 ', '용어사전', '지식베이스'],
      dtype='object')

# 각 column별 unique한 값 확인하기(생략)

# 고객의도가 '인증서'와 관련된 데이터만 필터링하기

In [ ]:
고객의도_리스트 = df['고객의도'].unique()
for idx, 고객의도 in enumerate(고객의도_리스트):
    print(f'{idx}: {고객의도}')

In [6]:
인증서_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '인증서' in 의도]
인증서_키워드_포함_고객의도_리스트

['인증서',
 '공인인증서발급문의',
 '공인인증서갱신',
 '카카오페이 인증서',
 '공인인증서암호',
 '인증서 연장',
 '인증서복사',
 '공인인증서',
 '공인인증서 연장',
 '공인인증서문의',
 '공인인증서오류',
 '공인인증서복사',
 '공인인증서만료',
 '기업공인인증서',
 '공인인증서발급',
 '공인인증서관리',
 '공인인증서 문의',
 '공인인증서 갱신',
 '타행인증서 등록',
 '공인인증서 연장신청',
 '모바일 인증서관리',
 '인증서 등록']

# 고객의도가 '거래내역조회'와 관련된 데이터만 필터링하기

In [7]:
거래내역조회_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '거래내역조회' in 의도]
거래내역조회_키워드_포함_고객의도_리스트

['거래내역조회',
 '거래내역조회문의',
 '계좌거래내역조회',
 '직원 거래내역조회',
 '통장거래내역조회',
 '스마트폰거래내역조회',
 '해외거래내역조회',
 '만기(해지)거래내역조회',
 '카드거래내역조회',
 'ISA 거래내역조회',
 '퇴직연금 거래내역조회',
 '보험 거래내역조회']

In [8]:
filtered_df = df[(df['고객의도'] == '거래내역조회') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
1232,금융/보험,잔고 및 거래내역,A4382,고객,1,거래내역조회,,Q,수고하십니다. 거래 내역 조회는 어디서 하나요?,,,,"수고, 거래, 내역",수고/노고,"거래,노고"
3236,금융/보험,잔고 및 거래내역,A5060,고객,1,거래내역조회,,Q,"거래내역을 확인하고 싶은데, 어떤 방법이 있는지 문의해도 되나요?",,,,"조건, 거래내역, 확인",거래내역/거래내용/ 확인/검증,"거래내역,검증"
3256,금융/보험,잔고 및 거래내역,A5061,고객,1,거래내역조회,,Q,설계사 지원금신청하려는데 거래내역 출력이 되나요?,,,,"설계사, 지원금신청, 제출, 거래내역, 출력",제출/지출/ 거래내역/거래내용/ 출력/프린트,"지원금신청,프린트"
3276,금융/보험,잔고 및 거래내역,A5062,고객,1,거래내역조회,,Q,2년전 거래내역 조회 문의하려구요,,,,"거래내역, 조회, 2년전, 내역",거래내역/거래내용/ 조회/검색/ 내역/내용,"조회,내용"
12962,금융/보험,잔고 및 거래내역,A7200,고객,1,거래내역조회,,Q,"OO은행 이용중인데요, 거래내역조회 어떻게 해요?",,,,"농협, 이용, 거래내역","농협/은행, 이용/사용, 거래내역/거래내용","이용,거래내용"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74257,금융/보험,잔고 및 거래내역,A32055,고객,1,거래내역조회,,Q,대출금 상환 내역 알려주세요,,,,"대출금,상환내역","대출금,상환내역,거래내역","상환내역,거래내역"
74277,금융/보험,잔고 및 거래내역,A32056,고객,1,거래내역조회,,Q,통장 거래내역 받아 볼 수 있나요?,,,,"통장,거래내역","통장,거래내역",거래내역
74297,금융/보험,잔고 및 거래내역,A32057,고객,1,거래내역조회,,Q,주식 매매거래 내역서 받아보고 싶어요,,,,"주식매매,거래내역","주식매매,거래내역,매매내역","거래내역,매매내역"
74318,금융/보험,잔고 및 거래내역,A32058,고객,1,거래내역조회,,Q,주식 매매 거래내역 알아보려구요,,,,"주식매매,거래내역","주식매매,거래내역,매매내역","거래내역,매매내역"


# 사용자 질문으로 부터 '고객요청의도' 추출하기

# OpenAI API Key 설정

In [9]:
OPENAI_KEY = "your_OpenAI_key"

In [10]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=OPENAI_KEY)

<ipython-input-10-5f82d27c8baf>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=OPENAI_KEY)


# create_tagging_chain으로 고객 요청의도 추출하기

In [11]:
from langchain.chains import create_tagging_chain

# 고객요청의도 추출을 위한 schema 설정
schema = {
    "properties": {
        "고객요청의도": {
            "type": "string",
            "enum": ['인증서',
                     '거래내역조회'
            ]
        }
    }
}
extraction_chain = create_tagging_chain(schema, llm)

<ipython-input-11-371e1159c934>:14: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` that is available on ChatModels capable of tool calling. See API reference for this function for replacement: <https://api.python.langchain.com/en/latest/chains/langchain.chains.openai_functions.tagging.create_tagging_chain.html> You can read more about `with_structured_output` here: <https://python.langchain.com/docs/how_to/structured_output/>. If you notice other issues, please provide feedback here: <https://github.com/langchain-ai/langchain/discussions/18154>
  extraction_chain = create_tagging_chain(schema, llm)


In [12]:
chain_run_result = extraction_chain.run("거래내역을 확인하고 싶은데, 어떤 방법이 있는지 문의해도 되나요?	")
chain_run_result

<ipython-input-12-8619405bb034>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain_run_result = extraction_chain.run("거래내역을 확인하고 싶은데, 어떤 방법이 있는지 문의해도 되나요?	")


{'고객요청의도': '거래내역조회'}

# 확인 생략

In [13]:
최종_필터링용_고객의도_리스트 = 거래내역조회_키워드_포함_고객의도_리스트 + 인증서_키워드_포함_고객의도_리스트
최종_필터링용_고객의도_리스트

['거래내역조회',
 '거래내역조회문의',
 '계좌거래내역조회',
 '직원 거래내역조회',
 '통장거래내역조회',
 '스마트폰거래내역조회',
 '해외거래내역조회',
 '만기(해지)거래내역조회',
 '카드거래내역조회',
 'ISA 거래내역조회',
 '퇴직연금 거래내역조회',
 '보험 거래내역조회',
 '인증서',
 '공인인증서발급문의',
 '공인인증서갱신',
 '카카오페이 인증서',
 '공인인증서암호',
 '인증서 연장',
 '인증서복사',
 '공인인증서',
 '공인인증서 연장',
 '공인인증서문의',
 '공인인증서오류',
 '공인인증서복사',
 '공인인증서만료',
 '기업공인인증서',
 '공인인증서발급',
 '공인인증서관리',
 '공인인증서 문의',
 '공인인증서 갱신',
 '타행인증서 등록',
 '공인인증서 연장신청',
 '모바일 인증서관리',
 '인증서 등록']

In [14]:
len(최종_필터링용_고객의도_리스트)

34

In [15]:
filtered_df = df[df['고객의도'].isin(최종_필터링용_고객의도_리스트)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,금융/보험,잔고 및 거래내역,A4091,고객,1,인증서,,Q,집과 직장에서 인터넷 뱅킹을 이용가능하나요?,,,,"가정, 직장, 인터넷 뱅킹",인터넷뱅킹/금융서비스,"직장,금융서비스"
2,금융/보험,잔고 및 거래내역,A4091,고객,3,인증서,,A,,,네,,,,
4,금융/보험,잔고 및 거래내역,A4091,고객,5,인증서,,Q,인증서 복사는 어떻게 하죠?,,,,"복사, 방법",,방법
6,금융/보험,잔고 및 거래내역,A4091,고객,7,인증서,,Q,이동식 USB말고 다른 방법있나요?,,,,"이동식USB, 방법",이동식 USB/이동식저장기기,"방법,이동식저장기기"
8,금융/보험,잔고 및 거래내역,A4091,고객,9,인증서,,A,,,네,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74348,금융/보험,잔고 및 거래내역,A32059,고객,11,거래내역조회,,A,,,00월00일 에 입금된 내역 조회부탁드려요,,"입금된,내역,조회","입금된,입금거래,내역,조회","내역,조회"
74351,금융/보험,잔고 및 거래내역,A32059,고객,14,거래내역조회,,Q,000이름으로 입금된 건이 있나요?,,,,"입금,조회","입금,입금거래내역,조회",조회
74353,금융/보험,잔고 및 거래내역,A32059,고객,16,거래내역조회,,Q,12명 총 입금액이 얼마인가요?,,,,"입금액,얼마","입금액,얼마,입금된금액","얼마,입금된금액"
74355,금융/보험,잔고 및 거래내역,A32059,고객,18,거래내역조회,,A,,,잘 알겠습니다.,,,,


# 중복 데이터 제거

In [ ]:
duplicates = filtered_df.duplicated(subset=filtered_df.columns.difference(['대화셋일련번호']), keep=False)

df_cleaned = filtered_df[~duplicates]
df_cleaned

In [ ]:
대화셋_set = set()
for count, (index, row) in enumerate(df_cleaned.iterrows()):
  print(count, row['대화셋일련번호'], row['고객질문(요청)'])
  대화셋_set.add(row['대화셋일련번호'])

In [ ]:
대화셋_set

In [ ]:
filtered_df = df[df['대화셋일련번호'].isin(대화셋_set)]
filtered_df

In [20]:
# '대화셋일련번호'로 그룹화하고 각 그룹을 인덱스 순서대로 정렬하여 원하는 컬럼들을 하나의 문자열로 합치기
def combine_columns(group):
  sorted_group = group.sort_index()
  combined_string = " ".join(
      sorted_group[['고객질문(요청)', '상담사질문(요청)', '고객답변', '상담사답변']].astype(str).values.flatten()
  )

  customer_intent = sorted_group['고객의도'].iloc[0]
  return pd.Series({'combined_string': combined_string, '고객의도': customer_intent})

In [ ]:
combined_result = filtered_df.groupby('대화셋일련번호').apply(combine_columns).reset_index()
combined_result

In [ ]:
# '고객요청의도' 컬럼 생성
def assign_customer_request_intent(intent):
  if intent in 거래내역조회_키워드_포함_고객의도_리스트:
    return '거래조회'
  elif intent in 인증서_키워드_포함_고객의도_리스트:
    return '인증서'
  else:
    return '기타'

combined_result['고객요청의도'] = combined_result['고객의도'].apply(assign_customer_request_intent)
combined_result

# Document 생성 및 Vectorstore에 Embedding해서 저장

In [23]:
from langchain.schema import Document

docs = []
인증서_docs = []
거래내역조회_docs = []

# 각 row의 값들을 순회하면서 출력
for index, row in combined_result.iterrows():

  doc = Document(page_content=row['combined_string'], metadata={'대화셋일련번호': row['대화셋일련번호'], '고객요청의도': row['고객요청의도']})
  docs.append(doc)

  if row['고객요청의도'] == '인증서':
    인증서_docs.append(doc)
  elif row['고객요청의도'] == '거래조회':
    거래내역조회_docs.append(doc)

In [ ]:
docs

In [25]:
len(docs)

144

In [ ]:
인증서_docs

In [27]:
len(인증서_docs)

33

In [ ]:
거래내역조회_docs

In [29]:
len(거래내역조회_docs)

101

# Embedding

In [30]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sroberta-multitask"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

<ipython-input-30-ff8597a753f8>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=docs, embedding=embedding_model, collection_name="all")

In [32]:
인증서_vectorstore = Chroma.from_documents(documents=인증서_docs, embedding=embedding_model, collection_name="certificate")

In [33]:
거래내역조회_vectorstore = Chroma.from_documents(documents=거래내역조회_docs, embedding=embedding_model, collection_name="transaction")

# Sanity Check

In [34]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
인증서_retriever = 인증서_vectorstore.as_retriever(search_kwargs={"k": 1})
거래내역조회_retriever = 거래내역조회_vectorstore.as_retriever(search_kwargs={"k": 1})

In [35]:
retrievered_docs = retriever.invoke(
    "집에서 말고 회사에서도 인터넷 뱅킹을 이용할 수 있을까요?"
)
print(retrievered_docs[0].page_content)

집에서 말고 회사에서도 인터넷 뱅킹을 이용할 수 있을까요?     본인이 발급받으신 공인인증서가 있으세요?     네 공인인증서는 집에 컴퓨터에 저장되어있어요     그럼 공인인증서를 복사하여 사용하는 방법을 안내해 드리겠습니다. 네 어떻게 하면 되죠?       댁내의 컴퓨터에 저장되어있는 공인인증서를 USB에 복사하여 사용하시는 방법이 있습니다. USB말고 다른 방법은 있을까요?       기기간에도 복사하는 방법이 있습니다.  안내해 드릴까요?     네 말씀해 주세요     인증서 내보내기 후에 암호를 입력하고  8자리 인증번호를 확인해 주세요.   네 확인했어요     그러면 인증서 가져오기를 누르시고 8자리 인증번호 입력해 주세요   네 입력했어요     인증서 복사가 완료되셨을꺼에요.    네 제 핸드폰으로 인증서 복사가 완료되었네요.   다른 문의 사항이 또 있으실까요?     아니요 다음에 생각나면 또 전화드릴께요.     네 고객님 상담은 이만 마치도록 하겠습니다.   수고하세요 


In [36]:
retrievered_docs = 인증서_retriever.invoke(
    "집에서 말고 회사에서도 인터넷 뱅킹을 이용할 수 있을까요?"
)
print(retrievered_docs[0].page_content)

집에서 말고 회사에서도 인터넷 뱅킹을 이용할 수 있을까요?     본인이 발급받으신 공인인증서가 있으세요?     네 공인인증서는 집에 컴퓨터에 저장되어있어요     그럼 공인인증서를 복사하여 사용하는 방법을 안내해 드리겠습니다. 네 어떻게 하면 되죠?       댁내의 컴퓨터에 저장되어있는 공인인증서를 USB에 복사하여 사용하시는 방법이 있습니다. USB말고 다른 방법은 있을까요?       기기간에도 복사하는 방법이 있습니다.  안내해 드릴까요?     네 말씀해 주세요     인증서 내보내기 후에 암호를 입력하고  8자리 인증번호를 확인해 주세요.   네 확인했어요     그러면 인증서 가져오기를 누르시고 8자리 인증번호 입력해 주세요   네 입력했어요     인증서 복사가 완료되셨을꺼에요.    네 제 핸드폰으로 인증서 복사가 완료되었네요.   다른 문의 사항이 또 있으실까요?     아니요 다음에 생각나면 또 전화드릴께요.     네 고객님 상담은 이만 마치도록 하겠습니다.   수고하세요 


In [37]:
retrievered_docs = 거래내역조회_retriever.invoke(
    "집에서 말고 회사에서도 인터넷 뱅킹을 이용할 수 있을까요?"
)
print(retrievered_docs[0].page_content)

OO은행 이용중인데요, 거래내역조회 어떻게 해요?     인터넷뱅킹 이용중이신가요?     이용 안하고 있는데요.     고객센터 이용방법 알려드리겠습니다. 요즘은 홈페이지랑 어플 많이 사용하잖아요 그런데는 사용못하나요?       홈페이지와 어플에서도 거래내역은 조회가능합니다. 홈페이지에서 조회하는 방법 알려주세요       홈페이지와어플은 인터넷뱅킹 가입자가 사용할 수 있습니다. 아! 인터넷뱅킹을 먼저 가입해야 되네요.. 지금 가입하고 조회하는 방법 알려주세요       인터넷 뱅킹은 영업점에 방문하셔야 합니다.  인터넷뱅킹 가입하시면서 거래내역서를 발급받으시면 빠르실 것 같습니다. 고객센터에서 신청해야겠네요 고객센터는 어떻게 이용해요?     이용중이신곳이 OO은행은행인가요?     OO은행은 그냥 OO은행아닌가요?   지금 이용중이신 통장 확인 하실 수 있나요?     네   통장 첫페이지를 보시면 OO은행은행이라고 적혀있나요?     네     OO은행은행 고객센터는 OOOO-OOOO 입니다. 이번호로 전화해서 거래내역 확인해 달라고 하면 되는거죠?       네 그렇습니다.


# Multiple Retriever를 활용하여 연관된 문서 찾아오기

In [38]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=OPENAI_KEY)

In [39]:
retrievers = {
    "": retriever,
    "인증서": 인증서_retriever,
    "거래내역조회": 거래내역조회_retriever,
}

In [40]:
from langchain_core.runnables import chain

In [41]:
@chain
def custom_chain(question):
  response = extraction_chain.run(question)
  print(response.get('고객요청의도', ""))
  retriever = retrievers[response.get('고객요청의도', "")]
  return retriever.invoke(question)

In [42]:
custom_chain.invoke("인증서 폐지는 언제부터 실시되나요?")

인증서


[Document(metadata={'고객요청의도': '인증서', '대화셋일련번호': 'A6636'}, page_content='공인인증서가 곧 폐지된다는데 그럼 본인인증을 어떻게 하나요?        기존의 공인인증서 대신 사설인증서를 사용하실 수 있습니다.  카카오페이의 인증서도 되나요?        네 됩니다.  어떻게 발급받나요?       카카오톡의 더보기 메뉴 중 자산 메뉴를 선택하세요.   네     서비스 메뉴의 인증을 선택하세요   네      약관에 동의하시고 휴대폰 본인인증을 하세요 끝인가요?        계좌인증을 하세요 제 계좌로 하나요?       네 계좌번호를 넣으면 되나요?       해당 은행으로 1원이 입금되는데 입금자명 4글자를 확인하세요.   확인했어요     입금자명을 카카오페이에 입력하시면 됩니다.    했어요     인증서 비밀번호를 설정하시면 됩니다.')]

# 대화 history 저장을 위한 ConversationBufferMemory 설정

In [43]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

<ipython-input-43-61e7e3748c65>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [44]:
memory.load_memory_variables({})

{'history': ''}

In [45]:
def load_memory(inpuut):
  return memory.load_memory_variables({})["history"]

# Tracking을 위한 LangSmith 설정 (Optional)

In [46]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"User Intent Analysis - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"


In [48]:
unique_id

'1234'

In [49]:
memory.load_memory_variables({})["history"]

''

In [50]:
def using_intent_get_relevant_document(user_question):
  history_combined_question = memory.load_memory_variables({})["history"] + "\nHuman" + user_question

  relevant_document = custom_chain.invoke(history_combined_question)[0]
  return relevant_document

In [51]:
relevant_document = using_intent_get_relevant_document("공인인증서 폐지는 언제부터 실시되나요?")
relevant_document

인증서


Document(metadata={'고객요청의도': '인증서', '대화셋일련번호': 'A6636'}, page_content='공인인증서가 곧 폐지된다는데 그럼 본인인증을 어떻게 하나요?        기존의 공인인증서 대신 사설인증서를 사용하실 수 있습니다.  카카오페이의 인증서도 되나요?        네 됩니다.  어떻게 발급받나요?       카카오톡의 더보기 메뉴 중 자산 메뉴를 선택하세요.   네     서비스 메뉴의 인증을 선택하세요   네      약관에 동의하시고 휴대폰 본인인증을 하세요 끝인가요?        계좌인증을 하세요 제 계좌로 하나요?       네 계좌번호를 넣으면 되나요?       해당 은행으로 1원이 입금되는데 입금자명 4글자를 확인하세요.   확인했어요     입금자명을 카카오페이에 입력하시면 됩니다.    했어요     인증서 비밀번호를 설정하시면 됩니다.')

#최종 Prompt 및 Chain 생성

In [53]:
from langchain.prompts import PromptTemplate

template = """다음과 같은 맥락과 채팅히스토리를 사용하여 마지막 질문에 대답하시오.
맥락: {context}
채팅히스토리: {history}
질문: {question}
도움이 되는 답변:"""
rag_prompt_custon = PromptTemplate.from_template(template)

In [55]:
# RAG chain 설정
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import MessagesPlaceholder

rag_chain = {"context": using_intent_get_relevant_document, "question": RunnablePassthrough(), "history": load_memory} | rag_prompt_custon | llm

In [56]:
def get_result_and_save_memory(user_question):
  chat_result = rag_chain.invoke(user_question)

  memory.chat_memory.add_user_message(user_question)
  memory.chat_memory.add_ai_message(chat_result.content)

  return chat_result

# OTP, 공인인증서 AI 고객센터 챗봇 성능 evaluation(정성적 평가)

## Case 1 - OTP

In [ ]:
temp_df = df[df['대화셋일련번호'] == 'A32051']
temp_df

In [59]:
llm_response = get_result_and_save_memory("금융거래 내역서 필요합니다.")
llm_response

거래내역조회


AIMessage(content='은행에서 금융거래 내역서를 발급받으시려면 신분증을 지참하시고 은행을 방문하셔야 합니다. 특정 입출금 내역, 예를 들어 한 사람에게 돈을 보낸 내역만 출력하는 것도 가능합니다. 이 경우 수수료는 2,000원입니다. 은행에 방문하셔서 필요한 내역을 요청하시면 됩니다.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 324, 'total_tokens': 411, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-0f4ed1e6-bb4a-496b-9084-8eda17c9b6b6-0')

In [60]:
memory.load_memory_variables({})["history"]

'Human: 금융거래 내역서 필요합니다.\nAI: 은행에서 금융거래 내역서를 발급받으시려면 신분증을 지참하시고 은행을 방문하셔야 합니다. 특정 입출금 내역, 예를 들어 한 사람에게 돈을 보낸 내역만 출력하는 것도 가능합니다. 이 경우 수수료는 2,000원입니다. 은행에 방문하셔서 필요한 내역을 요청하시면 됩니다.'

In [61]:
llm_response = get_result_and_save_memory("집에서 말고 회사에서도 인터넷 뱅킹을 이용할 수 있을까요?")
llm_response

거래내역조회


AIMessage(content='네, 인터넷 뱅킹은 집뿐만 아니라 회사에서도 이용할 수 있습니다. 인터넷이 연결된 컴퓨터나 모바일 기기에서 인터넷 뱅킹 웹사이트에 접속하여 로그인하시면 됩니다. 다만, 보안상의 이유로 공용 컴퓨터를 사용할 경우 로그아웃을 철저히 하고, 개인 정보가 유출되지 않도록 주의하시기 바랍니다.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 404, 'total_tokens': 489, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-ae85b628-40d8-4deb-9a20-84c1aff473bb-0')

In [62]:
memory.load_memory_variables({})["history"]

'Human: 금융거래 내역서 필요합니다.\nAI: 은행에서 금융거래 내역서를 발급받으시려면 신분증을 지참하시고 은행을 방문하셔야 합니다. 특정 입출금 내역, 예를 들어 한 사람에게 돈을 보낸 내역만 출력하는 것도 가능합니다. 이 경우 수수료는 2,000원입니다. 은행에 방문하셔서 필요한 내역을 요청하시면 됩니다.\nHuman: 집에서 말고 회사에서도 인터넷 뱅킹을 이용할 수 있을까요?\nAI: 네, 인터넷 뱅킹은 집뿐만 아니라 회사에서도 이용할 수 있습니다. 인터넷이 연결된 컴퓨터나 모바일 기기에서 인터넷 뱅킹 웹사이트에 접속하여 로그인하시면 됩니다. 다만, 보안상의 이유로 공용 컴퓨터를 사용할 경우 로그아웃을 철저히 하고, 개인 정보가 유출되지 않도록 주의하시기 바랍니다.'